In [10]:
import tensorflow as tf
import tensorflow_probability as tfp

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

tfd = tfp.distributions

tf.enable_eager_execution()

In [2]:
X=0
std=1
test = np.random.normal(loc=X, scale=std) 
print(test)

0.7038961033356856


In [19]:
def evaluate(tensors):
    if tf.executing_eagerly():
         return tf.contrib.framework.nest.pack_sequence_as(
             tensors,
             [t.numpy() if tf.contrib.framework.is_tensor(t) else t
             for t in tf.contrib.framework.nest.flatten(tensors)])
    with tf.Session() as sess:
        return sess.run(tensors)

In [22]:
challenger_data_ = np.array([[0,0,0],[1,1,1]])
temperature_ = challenger_data_
temperature = tf.convert_to_tensor(temperature_, dtype=tf.float32)
D_ = challenger_data_[:, 1]                # defect or not?
D = tf.convert_to_tensor(D_, dtype=tf.float32)
beta = tfd.Normal(name="beta", loc=0.3, scale=1000.).sample()
alpha = tfd.Normal(name="alpha", loc=-15., scale=1000.).sample()
p_deterministic = tfd.Deterministic(name="p", loc=1.0/(1. + tf.exp(beta * temperature_ + alpha))).sample()

print(p_deterministic)

[
    prior_alpha_,
    prior_beta_,
    p_deterministic_,
    D_,
] = evaluate([
    alpha,
    beta,
    p_deterministic,
    D,
])

print(prior_alpha_, prior_beta_)
print(p_deterministic_, D_)


tf.Tensor(
[[0. 0. 0.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
769.85626 -1304.6741
[[0. 0. 0.]
 [1. 1. 1.]] [0. 1.]


In [24]:
C = np.array([[0, 1],[1,0]])
X = np.random.rand(2, 100)
print(C.shape, X.shape)

XX = log_model_prob([X, C])
print(XX.shape)
# plt.plot(XX[0,:])

(2, 2) (2, 100)


TypeError: log_model_prob() missing 1 required positional argument: 'C'

In [ ]:
# Target distribution is proportional to: `exp(-x (1 + x))`.
def unnormalized_log_prob(x):
    print(x)
    return -x - x**2.


In [46]:
def leaky_if_model(X, C):
    X = tf.convert_to_tensor(X)
    C = tf.convert_to_tensor(C)
    return ReLU(tf.tensordot(C,X, axes=1)) - X

# def log_leakymodel(X, C):
#     return np.log(leaky_if_model(X, C))

def ReLU(x):
    return x * (x > 0)

def log_model_prob(X, C):
    rv_C = tfd.Uniform(name="conn_matrix", low=[0.0, 0.0, 0.0],
                                   high=[1., 1., 1., 1.])
    mean = leaky_if_model(X, C)
    std = tf.math.sqrt(X)
    # Define a single scalar Normal distribution.
    rv_observed = tfd.Normal(loc=mean, scale=std)
    
    return (
        rv_C.log_prob(C) +
        tf.reduce_sum(rv_observed.log_prob(X))
    )
#     return dist.log_prob
#     return np.random.normal(loc=X, scale=std, )    

In [47]:
#Create state to hold updated `step_size`.
step_size = tf.get_variable(
    name='step_size',
    initializer=1.,
    use_resource=True,  # For TFE compatibility.
    trainable=False)

In [48]:
print(step_size)
C = np.array([[0, 1],[1,0]], dtype=np.float32)

<tf.Variable 'step_size:0' shape=() dtype=float32, numpy=1.0>


In [49]:
# Initialize the HMC transition kernel.
num_results = int(10e3)
num_burnin_steps = int(1e3)
hmc = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=log_model_prob,
    num_leapfrog_steps=3,
    step_size=step_size,
    step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(
        num_adaptation_steps=int(num_burnin_steps * 0.8)))

# Run the chain (with burn-in).
samples, kernel_results = tfp.mcmc.sample_chain(
    num_results=num_results,
    num_burnin_steps=num_burnin_steps,
    current_state=[1, C],
    kernel=hmc)

IndexError: list index out of range

In [9]:
print(samples.shape)
# print(kernel_results.accepted_results)

(10000,)
UncalibratedHamiltonianMonteCarloKernelResults(log_acceptance_correction=<tf.Tensor 'mcmc_sample_chain/scan/TensorArrayStack_1/TensorArrayGatherV3:0' shape=(10000,) dtype=float32>, target_log_prob=<tf.Tensor 'mcmc_sample_chain/scan/TensorArrayStack_2/TensorArrayGatherV3:0' shape=(10000,) dtype=float32>, grads_target_log_prob=[<tf.Tensor 'mcmc_sample_chain/scan/TensorArrayStack_3/TensorArrayGatherV3:0' shape=(10000,) dtype=float32>])


In [5]:
# Initialize all constructed variables.
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    init_op.run()
    samples_, kernel_results_ = sess.run([samples, kernel_results])

print('mean:{:.4f}  stddev:{:.4f}  acceptance:{:.4f}'.format(
    samples_.mean(), samples_.std(), kernel_results_.is_accepted.mean()))
# mean:-0.5003  stddev:0.7711  acceptance:0.6240

mean:-0.5021  stddev:0.6979  acceptance:0.5029


In [ ]:
def leaky_if_model(X, C):
    return np.tanh(C.dot(X)) - X

In [4]:
logu = tf.
u = tfp.vi.kl_forward(
    logu,
    self_normalized=False,
    name=None
)

NameError: name 'logu' is not defined